Z wykorzystaniem modelu CreditMetrics wyznaczyć 99,9% VaR i ES dla portfela składającego się z portfela trzech poniżej opisanych obligacji skarbowych:

    3-letnia obligacja o ratingu A (cena wykupu 100 000 zł, subordinated),
    5-letnia obligacja o ratingu B (cena wykupu 50 000 zł, roczne kupony 5000 zł, senior secured),
    2-letnia obligacja o ratingu CCC (cena wykupu 50 000 zł, roczne kupony 10 000 zł, senior unsecured).

W badaniu założyć, że współczynniki korelacji między inwestycjami w portfelu wynoszą: r12 = 0.2, r13 = 0.15, r23 = 0.4.

Zbadać, w jaki sposób zmieniłyby się wyniki, jeśli założylibyśmy, że inwestycje w portfelu nie są skorelowane. Jak uzyskane wyniki łączą się z pojęciem "dywersyfikacji ryzyka".
Na podstawie dokumentacji technicznej CreditMetrics na przykładzie opisać, jakie są sposoby wyznaczania korelacji w modelu.

In [1]:
import numpy as np
import pandas as pd

In [2]:
#macierz przejścia dla ratingów
mp = np.array([
    [93.40, 5.94, 0.64, 0.00, 0.02, 0.00, 0.00, 0.00],
    [1.61, 90.55, 7.46, 0.26, 0.09, 0.01, 0.00, 0.02],
    [0.07, 2.28, 92.44, 4.63, 0.45, 0.12, 0.01, 0.00],
    [0.05, 0.26, 5.51, 88.48, 4.76, 0.71, 0.08, 0.15],
    [0.02, 0.05, 0.42, 5.16, 86.91, 5.91, 0.24, 1.29],
    [0.00, 0.04, 0.13, 0.54, 6.35, 84.22, 1.91, 6.81],
    [0.00, 0.00, 0.00, 0.62, 2.05, 4.08, 69.20, 24.05],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 100.00]
])
mp = mp/100
mp = pd.DataFrame(mp, columns = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Def'],
                  index = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Def'])

In [3]:
from scipy.stats import zscore
#stworzenie kumulatywnej macierzy przejścia
mp_cumulative = mp.cumsum(axis=1)
mp_cumulative = mp_cumulative.div(mp_cumulative.iloc[:, -1], axis=0)

In [11]:
import scipy.stats as st
mp_z = mp_cumulative.copy()
mp_z = 1 - mp_z
mp_z = -st.norm.ppf(mp_z)
mp_z = pd.DataFrame(mp_z, columns=mp.columns, index=mp.index)

In [31]:
[1,*mp_z.loc['A'][:]]


[1,
 -3.1946510537632724,
 -1.986300204129428,
 1.6248243080822067,
 2.5240846269919914,
 3.011453758499792,
 3.719016485455709,
 inf,
 inf]

In [5]:
#macierz stóp zwrotu ogółem
msz = pd.DataFrame([[3.60, 4.17, 4.73, 5.12],
                    [3.65, 4.22, 4.78, 5.17],
                    [3.72, 4.32, 4.93, 5.32],
                    [4.10, 4.67, 5.25, 5.63],
                    [5.55, 6.02, 6.78, 7.27],
                    [6.05, 7.02, 8.03, 8.52],
                    [15.05, 15.02, 14.03, 13.52]],
                    columns = ['1Y', '2Y', '3Y', '4Y'],
                    index = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC'])

In [6]:
#stopa odzysku według klasy obligacji
data = {'Mean': [53.80, 51.13, 38.52, 32.74, 17.09],
        'SD': [26.86, 25.45, 23.81, 20.18, 10.90]}
mso = pd.DataFrame(data,index = ["Senior Secured", "Senior Unsecured",
                                 "Senior Subordinated", "Subordinated", "Junior Subordinated"])

In [7]:
mcorr = pd.DataFrame([[1.0, 0.2, 0.15],
                      [0.2, 1.0, 0.4],
                      [0.15, 0.4, 1.0]],
                      columns=['r1', 'r2', 'r3'],
                      index=['r1', 'r2', 'r3'])

In [8]:
valA = 100000 #subordinated
valB = 50000 #senior secured
couponB = 5000
valCCC = 50000 #senior unsecured
couponCCC = 10000

In [45]:
import numpy as np
# Symulacja scenariuszy
np.random.seed(123)
num_scenarios = 1000
scenarios = np.random.multivariate_normal(mean=np.zeros(len(mcorr)), cov=mcorr, size=num_scenarios)
scenarios = pd.DataFrame(scenarios, columns=["Obligacja1","Obligacja2","Obligacja3"])

#ten zakres odpowiada jedynie przejściom z BBB, trzeba to rozszerzyć na wszystkie możliwe
value_rangeA = [-float('inf'),-3.19, -1.99, 1.63, 2.52, 3.01, 3.72, 3.73, float('inf')]
value_rangeB = [-float('inf'),-3.5, -3.35, -2.93, -2.45, -1.47, 1.36, 1.49, float('inf')]
value_rangeCCC = [-float('inf'),-4, -3.5, -3, -2.50, -1.93, -1.49, 0.70, float('inf')]
letters = ['Def', 'CCC', 'B', 'BB', 'BBB', 'A', 'AA', 'AAA']
letters = letters[::-1]

# scenarios['Obl1'] = pd.cut(scenarios['Obligacja1'], bins=np.unique([-float('inf'),*mp_z.loc['A'].values]), labels=letters[:-2])
# scenarios['Obl2'] = pd.cut(scenarios['Obligacja2'], bins=np.unique([-float('inf'),*mp_z.loc['B'].values]), labels=letters[1:-1])
# scenarios['Obl3'] = pd.cut(scenarios['Obligacja3'], bins=np.unique([-float('inf'),*mp_z.loc['CCC'].values]), labels=letters[3:])
scenarios['OblA'] = pd.cut(scenarios['Obligacja1'], bins=value_rangeA, labels=letters)
scenarios['OblB'] = pd.cut(scenarios['Obligacja2'], bins=value_rangeB, labels=letters)
scenarios['OblCCC'] = pd.cut(scenarios['Obligacja3'], bins=value_rangeCCC, labels=letters)
scenarios = scenarios[['OblA', 'OblB', 'OblCCC']]

sprawdzamy obligacje za rok, czyli mając obligacje 2,5 i 3 letnie będziemy mieli obligacje 1,4 i 2 letnie
wycena polega na tym, że dyskontujemy wszystkie przepływy finansowe na dany moment w czasie
jeśli jest bankrut to losujemy z rozkładu N(m,s) stopy odzysku